In [1]:
import time
import random
from sklearn.ensemble import GradientBoostingClassifier
time_start = time.time()

In [2]:
buyset = {}
for line in sc.textFile('./buy').collect():
    buyset[line.strip()] = True
X_train_0, Y_train_0, X_train_1, Y_train_1 = [], [], [], []

In [3]:
dates = ["12-17-0","12-18-0"]
# dates = ["12-19-0"]
for date in dates:
    for line in sc.textFile('./prediction/'+date, 1).collect():
        if buyset.has_key(date+"\t"+"\t".join(line.strip().split("\t")[:2])):
            X_train_1.append([float(line.strip().split("\t")[3:][i]) for i in xrange(len(line.strip().split("\t")[3:])) if i in [1,2,4,5,9,12,15,17,18,20,23,27,48,50,62,66]])
            Y_train_1.append(1)
        else:
            X_train_0.append([float(line.strip().split("\t")[3:][i]) for i in xrange(len(line.strip().split("\t")[3:])) if i in [1,2,4,5,9,12,15,17,18,20,23,27,48,50,62,66]])
            Y_train_0.append(0) 

In [4]:
N = 3
X_train, Y_train = [[] for i in xrange(N)], [[] for i in xrange(N)]
for i in xrange(N):
        for j in random.sample([j for j in xrange(len(X_train_0))], 10*len(X_train_1)):
            X_train[i].append(X_train_0[j])
            Y_train[i].append(Y_train_0[j])
        X_train[i].extend(X_train_1)
        Y_train[i].extend(Y_train_1)

In [5]:
print X_train[0][0]
print Y_train[0][0]
print len(Y_train[0])

[100.0, 3.0, 0.0, 3.0, 8.0, 2.0, 1.0, 5.0, 7.0, 0.0, 100.0, 3.0, 3.0, 0.0, 0.0, 0.0]
0
4290


In [5]:
clfs = [GradientBoostingClassifier(n_estimators=50, max_depth=3, learning_rate=0.1) for i in xrange(N)]

In [6]:
for i in xrange(N):
    clfs[i].fit(X_train[i],Y_train[i])
print "training finished!~"

training finished!~


In [7]:
def prediction(line):
    feature = [float(line.strip().split("\t")[3:][i]) for i in xrange(len(line.strip().split("\t")[3:])) if i in [1,2,4,5,9,12,15,17,18,20,23,27,48,50,62,66]]
    return (sum([clfs[i].predict_proba([feature])[0][0] for i in xrange(N)]), "\t".join(line.strip().split("\t")[:2]))

In [8]:
pred = "12-19-0"
lines = sc.textFile('./prediction/'+pred, 1)
counts = lines.map(lambda x : prediction(x)).sortByKey().collect()
# recall, precision, correct = len([1 for key in buyset.keys() if key.strip().split("\t")[0] == pred]), 400, len([1 for item in counts[:400] if buyset.has_key(pred+"\t"+item[1])])
recall, precision, correct = len([1 for key in buyset.keys() if key.strip().split("\t")[0] == pred]), len(counts), len([1 for item in counts if buyset.has_key(pred+"\t"+item[1])])
print "prediction finished!~"

prediction finished!~


In [9]:
print recall, precision, correct, 1.0*correct/recall, 1.0*correct/precision, 2*1.0*correct/recall*1.0*correct/precision/(1.0*correct/recall+1.0*correct/precision)

0 148385 0

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/tmp/kernel-PySpark-1ee3abcb-336a-44d5-8b89-fc4f80945260/pyspark_runner.py", line 189, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
ZeroDivisionError: float division by zero

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:162)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.reflection.R

In [10]:
time_end = time.time()
print time_end-time_start

 134.337442875


In [20]:
for i in range(20):
    print counts[i][1].split()

[u'35482754', u'307600160']
[u'138961180', u'98603220']
[u'25737649', u'17712290']
[u'100226515', u'250781249']
[u'5369231', u'394658338']
[u'134979525', u'338630082']
[u'50114359', u'283687321']
[u'122132057', u'288398671']
[u'62879735', u'384038335']
[u'31182685', u'332581378']
[u'50114359', u'229347739']
[u'137640381', u'205571401']
[u'31182685', u'73233753']
[u'43484740', u'222614344']
[u'103823974', u'37910916']
[u'128338662', u'311499944']
[u'48967362', u'68940123']
[u'10129914', u'249618450']
[u'4915814', u'343575787']
[u'138780010', u'401070714']


In [29]:
user_id, item_id = [], []
for i in range(len(counts)):
#     print (counts[i][1].split())[0], (counts[i][1].split())[1]
    user_id.append((counts[i][1].split())[0])
    item_id.append((counts[i][1].split())[1])

In [30]:
len(user_id), len(item_id)

(148385, 148385)

In [13]:
len(counts)

148385

In [14]:
import pandas as pd

In [31]:
sub = pd.DataFrame()

In [32]:
sub['user_id'] = user_id

In [33]:
sub['item_id'] = item_id

In [35]:
sub.to_csv('tianchi_mobile_recommendation_predict.csv', index = False)

In [34]:
sub

          user_id    item_id
0        35482754  307600160
1       138961180   98603220
2        25737649   17712290
3       100226515  250781249
4         5369231  394658338
5       134979525  338630082
6        50114359  283687321
7       122132057  288398671
8        62879735  384038335
9        31182685  332581378
10       50114359  229347739
11      137640381  205571401
12       31182685   73233753
13       43484740  222614344
14      103823974   37910916
15      128338662  311499944
16       48967362   68940123
17       10129914  249618450
18        4915814  343575787
19      138780010  401070714
20      112619968   95867699
21      121986074  215620055
22       38026943  253800192
23       35482754  399454553
24       61750058   67897568
25       18249259   80411699
26       12952941  195290669
27       13709817  299941043
28       41686678  385753960
29       12952941  401863214
...           ...        ...
148355   52685207  276983475
148356   52685207  126065135
148357   52685